In [ ]:
# done in kaggle notebook
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

2026-01-22 04:59:49.556937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769057989.795074      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769057989.860590      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769057990.413329      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769057990.413381      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769057990.413384      55 computation_placer.cc:177] computation placer alr

In [2]:
train_dir = "/kaggle/input/intel-image-classification/seg_train/seg_train"
test_dir  = "/kaggle/input/intel-image-classification/seg_test/seg_test"

In [3]:
# Training generator with augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

# Test/validation generator (no augmentation, only rescale)
test_gen = ImageDataGenerator(rescale=1./255)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # CRITICAL for metrics
)


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 classes for Intel dataset
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-22 05:00:12.449408: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,829,126 (18.42 MB)

 Trainable params: 4,829,126 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 363s 821ms/step - accuracy: 0.4640 - loss: 1.3431 - val_accuracy: 0.5960 - val_loss: 1.0752
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 344s 783ms/step - accuracy: 0.6446 - loss: 0.9384 - val_accuracy: 0.7530 - val_loss: 0.6970
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 336s 766ms/step - accuracy: 0.7014 - loss: 0.8085 - val_accuracy: 0.7407 - val_loss: 0.7348
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 337s 768ms/step - accuracy: 0.7443 - loss: 0.7212 - val_accuracy: 0.7910 - val_loss: 0.5941
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 389s 785ms/step - accuracy: 0.7749 - loss: 0.6670 - val_accuracy: 0.7997 - val_loss: 0.5980
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 377s 774ms/step - accuracy: 0.7868 - loss: 0.6087 - val_accuracy: 0.8180 - val_loss: 0.5101
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 323s 736ms/step - accuracy: 0.7969 - loss: 0.5795 - val_accuracy: 0.8133 - val_loss: 0.5304
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 322s 732ms/step - accuracy: 0.8036 -

In [6]:
# Predict on test data
val_steps = test_data.samples // test_data.batch_size + 1
pred_probs = model.predict(test_data, steps=val_steps)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_data.classes
class_names = list(test_data.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_names))

94/94 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step
              precision    recall  f1-score   support

   buildings       0.86      0.82      0.84       437
      forest       0.95      0.98      0.96       474
     glacier       0.87      0.72      0.79       553
    mountain       0.84      0.72      0.78       525
         sea       0.72      0.95      0.82       510
      street       0.85      0.88      0.87       501

    accuracy                           0.84      3000
   macro avg       0.85      0.84      0.84      3000
weighted avg       0.85      0.84      0.84      3000



In [7]:
model.save("intel_cnn_model.keras")

In [8]:
import os
os.listdir(".")


['intel_cnn_model.keras', '.virtual_documents']